# 🌾 KCC Chatbot with ChromaDB, MPNet & Gemma

In [112]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb import PersistentClient
from ollama import Client as OllamaClient
from duckduckgo_search import DDGS
import time

In [113]:
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

client = PersistentClient(path="./chroma_db")
try:
    collection = client.get_collection("kcc_farm_qa")
except:
    collection = client.create_collection("kcc_farm_qa")

ollama_client = OllamaClient()

In [114]:
def query_chromadb(user_query, top_k=3):
    biased_query = f"According to farming query: {user_query}"
    query_embedding = embedder.encode([biased_query])[0].tolist()
    
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k * 2)
    
    documents = results.get('documents', [[]])[0]
    distances = results.get('distances', [[]])[0]
    
    good_docs = []
    for doc, distance in zip(documents, distances):
        if distance < 0.8: 
            good_docs.append(doc)
    
    return good_docs[:top_k]

def ask_model(prompt):
    
    response = ollama_client.chat(model='gemma3:1b', messages=[
        {"role": "user", "content": prompt}
    ])
    return response['message']['content']

def fallback_search(query, max_results=2):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        return [r["body"] for r in results]

def query_chatbot(user_query, top_k=3):
    docs = query_chromadb(user_query, top_k=top_k)
    print(f"Found {len(docs)} good documents")

    for i,n in enumerate(docs):
        print(f"{i+1}:{n}")

    if docs:
        print("\n✅ USING LOCAL DB")
        context = "\n\n".join([f"Context {i+1}: {d}" for i, d in enumerate(docs)])
        prompt= f"""
        You are given some documents. First, **translate the following documents into English** and display them under the header **"Translations:"**.

        Then, using **only** the translated content and the provided context, write a **concise one-paragraph answer** to the question in **English**. Begin with a brief summary of all relevant context, then answer the question clearly and completely. Do not include any information not found in the documents or context.
        ---

        Documents:
        {docs}

        Context:
        {context}

        Question:
        {user_query}

        ---

        Translations:
        (Translate the documents in English here and seperate the Question, Regional Language Answer, Translated Answer)

        Answer:
        (Write the answer here)
        """
    else:
        print("\n⚠️ Searching online...")
        fallback = fallback_search(user_query)
        context = "\n\n".join([f"Web Result {i+1}: {d}" for i, d in enumerate(fallback)])
        prompt = f"""
        Using only the provided context, write a **concise one-paragraph answer** to the question in **English**. Begin with a brief summary of the relevant context, then answer the question clearly and completely. Do not include any information that is not in the context.

        ---

        Context:
        {context}

        Question:
        {user_query}

        ---

        Answer:
        (Write the answer here)
        """
    try:
        return ask_model(prompt)
    except Exception as e:
        return f"Error: {e}"

In [115]:
TOP_K_DOCUMENTS=3
reply = query_chatbot("What issues do sugarcane farmers in Maharashtra commonly face?", TOP_K_DOCUMENTS)
print(f"🤖 {reply}\n")
reply = query_chatbot("Where is Atlantis located", TOP_K_DOCUMENTS)
print(f"🤖 {reply}\n")

Found 3 good documents
1:FARMER ASKED ABOUT INFORMATION OF WEED MANAGEMENT IN SUGARCANE CROP ? शेतकरी साहेब, आपल्या ऊस पिकातील तानियंत्रणासाठी   खुरपणी करून घ्यावी. धन्यवाद...
2:FARMER ASKED ABOUT CULTIVATION OF SUGARCANE CROP WITHOUT TILLAGE OPERATION? जमीन भुसभुशीत असेल तर जमिनीची मशागत न करता आपण ऊस पिकाची लागवड करू शकता
3:WEED MANAGEMENT IN SUGARCANE ? ऊस -तण नाशक - त्रिशुक 1200 ग्रॅम 200 लिटर पाण्यासाठी घेऊन फवारणी करावी धन्यवाद

✅ USING LOCAL DB
🤖 Translations:
'FARMER ASKED ABOUT INFORMATION OF WEED MANAGEMENT IN SUGARCANE CROP ? शेतकरी साहेब, आपल्या ऊस पिकातील तानियंत्रणासाठी   खुरपणी करून घ्यावी. धन्यवाद...'
'FARMER ASKED ABOUT CULTIVATION OF SUGARCANE CROP WITHOUT TILLAGE OPERATION? जमीन भुसभुशीत असेल तर जमिनीची मशागत न करता आपण ऊस पिकाची लागवड करू शकता'
'WEED MANAGEMENT IN SUGARCANE ? ऊस -तण नाशक - त्रिशुक 1200 ग्रॅम 200 लिटर पाण्यासाठी घेऊन फवारणी करावी धन्यवाद'

Answer:
Sugarcane farmers in Maharashtra commonly face weed management issues, requiring frequent and precise ha